In [6]:
import re
import jieba.posseg as posseg
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

###################################################
# TextRank实现
###################################################


# 停用词路径
stopwords_path = 'stopwords.txt'
# 需要排除的词性
stopPOS = []

# 读取停用词
with open(stopwords_path, 'r', encoding='utf-8') as f:
    stopwords = [line.strip() for line in f.readlines()]

def segment_text_to_sentence(text):
    # 将文本分割成句子
    sentences = re.split(r'[。！？!?]', text)
    sentences = [sentence.strip().replace(" ", "").replace('\n','') for sentence in sentences if sentence.strip()]
    return sentences

def segment_text_to_words(text, use_stopwords):
    # 分词并去除停用词
    global stopPOS, stopwords
    stopPOS = [item.lower() for item in stopPOS]
    words = posseg.cut(text)
    if use_stopwords:
        words = [word for word, flag in words if flag[0].lower() not in stopPOS and word not in stopwords]
    else:
        words = [word for word, flag in words if flag[0].lower() not in stopPOS]
    words = set(words)

    return words

def original_similarity_matrix(sentences, use_stopwords):
    # 计算原始相似性矩阵
    sentence_words = [set(segment_text_to_words(item, use_stopwords)) for item in sentences]
    size = len(sentences)
    similarity_matrix = np.zeros((size, size))
    for i in range(size):
        for j in range(i+1, size):
            if len(sentence_words[i]) == 0 or len(sentence_words[j]) == 0:
                similarity = 0
            else:
                # 计算相似性
                similarity = len(sentence_words[i] & sentence_words[j]) / (np.log(len(sentence_words[i])) + np.log(len(sentence_words[i])) + 1e-10)
            similarity_matrix[i][j] = similarity_matrix[j][i] = similarity
    return similarity_matrix

def cosine_tfidf_similarity_matrix(sentences, use_stopwords):
    # 计算基于TF-IDF的余弦相似性矩阵
    sentence_words = [' '.join(segment_text_to_words(item, use_stopwords)) for item in sentences]
    tfidf_vectorizer = TfidfVectorizer()
    tfidf_matrix = tfidf_vectorizer.fit_transform(sentence_words)
    similarity_matrix = cosine_similarity(tfidf_matrix)
    
    # 将对角线元素设置为0，避免自身与自身的相似性干扰
    np.fill_diagonal(similarity_matrix, 0)
    return similarity_matrix

def summarize_text_rank(text, d=0.85, iter_num=200, top=3, method='默认方式', use_stopwords=True):
    sentences = segment_text_to_sentence(text)

    print('---------开始----------------------------------------')
    if method == '默认方式':
        edge_weight = original_similarity_matrix(sentences, use_stopwords)
    elif method == 'TF-IDF':
        edge_weight = cosine_tfidf_similarity_matrix(sentences, use_stopwords)
        
    node_weight = np.ones((len(sentences)))

    for num in range(iter_num):                
        # TextRank迭代公式
        node_weight_new = (1-d) + d * node_weight @ (edge_weight / (edge_weight.sum(axis=-1) + 1e-10)).T
        if ((node_weight_new - node_weight)**2).sum() < 1e-10:
            break
        node_weight = node_weight_new

    if num < iter_num:
        print('迭代{}次，收敛'.format(num))
    else:
        print('迭代{}次，未收敛'.format(num))

    sorted_indices = np.argsort(node_weight)[::-1]

    # 获取最大的几个值及其对应的索引
    top_indices = sorted(sorted_indices[:top])
    top_values = node_weight[top_indices]

    #print('最大的{}个值：'.format(top), top_values)
    #print('对应的索引：', top_indices)
    #print('结果：')
    result = ''
    for idx in top_indices:
        result += sentences[idx] + '。\n'
    print(result)

    return result

text = '界面新闻记者|戈振伟界面新闻编辑|林腾位于中东的黎巴嫩在9月连续发生了大规模的寻呼机和对讲机爆炸事件，致使供应链安全成为全球关注焦点。社交媒体上有传言称，出于安全考虑，中东的通讯类电子产品订单正大规模往中国大陆转移，有的甚至列出了具体的产品数量和订单金额，一些中国大陆厂商据说要开始通过加班来完成订单交付。界面新闻记者主要以对讲机这个品类就上述传言进行调查了解后发现，目前并没有直接证据表明中东的通讯类电子产品订单正在大规模向中国大陆转移，多位受访的对讲机供应商表示来自中东的订单没有明显变化，网上的传言有所夸大，有专家认为中东的电子订单即使部分转移到中国大陆，影响也很小。黎巴嫩爆炸事件之后，订单转移的风第一时间吹到了深圳华强北。华强北有“中国电子第一街”之称，是亚洲规模最大的电子产品集散地，市场嗅觉敏锐。传闻称：“因为以色列引爆了传呼机，引爆了其它的电子产品等，给中东和其他国家造成了极大的恐慌，现在华强北突然爆单，很多产品要求在中国全过程生产，不得使用第三国元器件。”一位在华强北卖了多年对讲机的老板告诉记者：“最近是多了一些中东的中间商过来询价，每天大概四五个（之前一天一两个），不过还没有正式的订单成交。”他认为目前网上的一些传言夸大了。另外，该老板告诉记者，华强北早就不卖传呼机。华强北研究者、萨科微和金航标总经理宋仕强表示：“传呼机也叫BB机，是过去了20年的通信产品，市场上已经看不到了，在华强北也没有相关白色和灰色的产业链。”他认为，华强北主打的是电子元器件，是传呼机等消费品的零部件，中间差好几个环节，“华强北的效率非常高，但短时间华强北的产业链还串不起来”。另一位华强北长期从业者陈实（化名）对记者称：“黎巴嫩爆炸这个事对华强北是有利好，但不像网传的那样。”他认为整个利好对华强北的元器件市场来说基本等于1%都没有。陈实分析，中东国家来华强北采购是很早就有的事了，只不过这次有人借传呼机爆炸做话题，毕竟这种话题只要不伤害到别人，现阶段有可能是个增加营收的机会，但订单体量也没有网传那么大。记者注意到，截至10月11日，网上之前一些订单转移的短视频和自媒体文章已经被下架。不过，资本市场对此的热度仍在持续，其中最受关注的是对讲机龙头海能达（002583.SZ）。市场有观点认为，海能达会成为这波订单转移的最大受益者之一。海能达成立于1993年，总部位于深圳。数据显示，2023年海能达的对讲机全球占比11.41%，位列第二。排名第一的是摩托罗拉，占比为58.57%。海能达股价自黎巴嫩爆炸事件之后就连续拉了多个涨停，不到一个月市值已经翻倍。公司近期发布了三次股票交易异常波动公告，称“公司未发现近期公共传媒报道了可能或已经对公司股票交易价格产生较大影响的未公开重大信息”。记者在海能达龙岗工厂门口发现，海能达在9月20日开启了新一轮招聘，入职还发放1000元左右的奖励。就订单转移一事，记者进一步向海能达方面求证，对方未予置评。同样，网上传闻涉及的华为、中兴也未回应记者的置评请求。深圳之外，记者还致电了多位泉州的对讲机生产商和代理商，均未获得近期中东对讲机订单明显增长的回复。泉州是全国对讲机产品生产和销售的集散中心，有“对讲机之乡”之称，80年代初，全国第一台国产对讲机在泉州市诞生。事实上，海能达创始人陈清州便来自泉州。福建省工业和信息化厅数据显示，2022年，泉州市共有200多家对讲机生产企业，年产对讲机约4500万部，产量占全国约70%。此外，据财联社，多家通讯电子类上市公司、元器件分销商近期并未明显感受到来自中东地区的订单。中东地区包括黎巴嫩、以色列、伊拉克、土耳其、沙特阿拉伯等23个国家与地区，约4.9亿人口，以石油资源丰富著称，是目前世界上石油储量最大、生产石油最多的地区。截至2023年底，中东地区的GDP达49030.6亿美元，GDP占全球比重为5.3%。以经济规模来看，2023年GDP排名前五的中东地区国家依次是土耳其、沙特阿拉伯、伊朗、埃及、阿联酋，这五大国家合计GDP占该地区比重达70.4%，其中比重超过15%的只有土耳其和沙特阿拉伯。中东地区的电子工业薄弱，电子产品主要依靠进口。2023年，中国向中东地区出口金额达2529.3亿美元，其中电子电机电气设备及零件是近年来首要的出口品类，份额占据近五分之一，其中土耳其、阿联酋等份额较大。电信设备分销商ZMC集团伊拉克GM曾琪告诉界面新闻记者：“除了土耳其、阿联酋等国家从中国进口物料在本地搞一些自有品牌，其他中东国家基本没有什么电子工业（没有产业链基础、基础设施优势和价格优势），基本都依赖从中日韩和西方进口，消费级的电子产品，主要还是中国来的，加上一些西方国家的大品牌。”以ZMC举例，ZMC近期在伊拉克开办了一个手机组装厂，成为中东（不含土耳其、北非）第一家智能手机组装厂，当地有很多媒体报道，也引起了当地民众的热议。据曾琪观察，随着中国众多一线品牌出海，而且占据了很大的市场份额，中东消费者对中国电子产品的认知，相比十几年前有了质的变化。中东消费者眼中的中国品牌价值和品质早已今非昔比，中国制造影响着他们生活的方方面面。由于产业链的优势，中国是消费级电子产品的主要整机集成商，美欧日韩在很多领域成了背后核心部件的供应商，“在消费市场上面，我并没有听到多少关于订单转移到中国的消息，市场基本还是比较稳定”。“当然，不排除来自中东国家的政府订单增加了，或者相对特殊领域的消费产品订单转移到中国了，这个我还缺乏案例。”曾琪补充道。国家之间的关系，确实会影响到民众对产品的认知和信任。由于西方品牌和文化在中东沉淀时间也很长，曾琪称目前处在非战争的国家的民众还是比较理性，不太可能出现一边倒的情况。电子供应链长期观察者、深圳市软科学研究会常务副会长金心异对记者分析，中东个别国家因为国际制裁或各种安全原因来增加对中国电子产品的采购是有可能，但这不是一个趋势性现象，是个偶然现象，况且中东这些国家的电子产品市场本来就不大，对深圳而言，这块市场还是比较小，影响有限。界面新闻记者陆柯言对本文亦有贡献'

summarize_text_rank(text)

---------开始----------------------------------------
迭代8次，收敛
界面新闻记者主要以对讲机这个品类就上述传言进行调查了解后发现，目前并没有直接证据表明中东的通讯类电子产品订单正在大规模向中国大陆转移，多位受访的对讲机供应商表示来自中东的订单没有明显变化，网上的传言有所夸大，有专家认为中东的电子订单即使部分转移到中国大陆，影响也很小。
电信设备分销商ZMC集团伊拉克GM曾琪告诉界面新闻记者：“除了土耳其、阿联酋等国家从中国进口物料在本地搞一些自有品牌，其他中东国家基本没有什么电子工业（没有产业链基础、基础设施优势和价格优势），基本都依赖从中日韩和西方进口，消费级的电子产品，主要还是中国来的，加上一些西方国家的大品牌。
电子供应链长期观察者、深圳市软科学研究会常务副会长金心异对记者分析，中东个别国家因为国际制裁或各种安全原因来增加对中国电子产品的采购是有可能，但这不是一个趋势性现象，是个偶然现象，况且中东这些国家的电子产品市场本来就不大，对深圳而言，这块市场还是比较小，影响有限。



'界面新闻记者主要以对讲机这个品类就上述传言进行调查了解后发现，目前并没有直接证据表明中东的通讯类电子产品订单正在大规模向中国大陆转移，多位受访的对讲机供应商表示来自中东的订单没有明显变化，网上的传言有所夸大，有专家认为中东的电子订单即使部分转移到中国大陆，影响也很小。\n电信设备分销商ZMC集团伊拉克GM曾琪告诉界面新闻记者：“除了土耳其、阿联酋等国家从中国进口物料在本地搞一些自有品牌，其他中东国家基本没有什么电子工业（没有产业链基础、基础设施优势和价格优势），基本都依赖从中日韩和西方进口，消费级的电子产品，主要还是中国来的，加上一些西方国家的大品牌。\n电子供应链长期观察者、深圳市软科学研究会常务副会长金心异对记者分析，中东个别国家因为国际制裁或各种安全原因来增加对中国电子产品的采购是有可能，但这不是一个趋势性现象，是个偶然现象，况且中东这些国家的电子产品市场本来就不大，对深圳而言，这块市场还是比较小，影响有限。\n'